In [5]:
!pip install -q torch transformers transformers accelerate datasets bitsandbytes attacut langchain sentence-transformers openpyxl pacmap ragatouille matplotlib 
!pip install -q -U langchain-community

  DEPRECATION: Building 'annoy' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'annoy'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'docopt' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'docopt'. Discussion can be found at https://github.com/pypa/pip/issues/6334
ERROR: pip's dependency resolver does not currently take into account all the packages tha

In [43]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset
import matplotlib.pyplot as plt

pd.set_option(
    "display.max_colwidth", None
)  # this will be helpful when visualizing retriever outputs

In [44]:
# Import data
import requests
from datasets import Dataset
from io import StringIO

def load_first_500_samples(url):
    """
    Load only the first 500 rows from ThaiSum dataset efficiently
    """
    print("📥 Loading first 500 samples from ThaiSum dataset...")
    print(f"🌐 URL: {url}")
    
    try:
        # Use streaming to read only what we need
        print("🔄 Starting streaming download...")
        
        response = requests.get(url, stream=True, timeout=60)
        response.raise_for_status()
        
        print(f"✅ Connection successful! Status: {response.status_code}")
        
        # Read line by line until we get 501 lines (header + 500 rows)
        lines = []
        line_count = 0
        
        print("📖 Reading first 500 rows...")
        
        # Decode content and split into lines
        content = ""
        for chunk in response.iter_content(chunk_size=8192, decode_unicode=True):
            if chunk:
                content += chunk
                
                # Process complete lines
                while '\n' in content:
                    line, content = content.split('\n', 1)
                    lines.append(line + '\n')
                    line_count += 1
                    
                    # Stop when we have header + 500 data rows
                    if line_count >= 501:
                        break
                
                if line_count >= 501:
                    break
        
        print(f"📊 Read {line_count} lines (including header)")
        
        # Join lines and create CSV content
        csv_content = ''.join(lines)
        
        # Read with pandas
        print("🔄 Converting to DataFrame...")
        df = pd.read_csv(StringIO(csv_content))
        
        print(f"✅ Successfully loaded {len(df)} rows!")
        print(f"📋 Columns: {list(df.columns)}")
        
        # Convert to Hugging Face Dataset
        print("🔄 Converting to Hugging Face Dataset...")
        dataset = Dataset.from_pandas(df)
        
        print("🎉 Dataset ready!")
        
        # Show sample
        print("\n" + "="*60)
        print("📄 FIRST SAMPLE:")
        print("="*60)
        sample = dataset[0]
        
        for key, value in sample.items():
            print(f"\n{key.upper()}:")
            if isinstance(value, str):
                if len(value) > 150:
                    print(f"  {value[:150]}...")
                    print(f"  [Length: {len(value)} characters]")
                else:
                    print(f"  {value}")
            else:
                print(f"  {value}")
        
        # Show basic stats
        if 'body' in dataset.column_names and 'summary' in dataset.column_names:
            print(f"\n📈 QUICK STATS:")
            bodies = [len(str(dataset[i]['body']).split()) for i in range(min(50, len(dataset)))]
            summaries = [len(str(dataset[i]['summary']).split()) for i in range(min(50, len(dataset)))]
            
            print(f"Average body length: {sum(bodies)/len(bodies):.1f} words")
            print(f"Average summary length: {sum(summaries)/len(summaries):.1f} words")
        
        return dataset
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None

# Even simpler approach using pandas directly
def load_simple_pandas(url, n_rows=500):
    """
    Simplest approach - let pandas handle it
    """
    print(f"📥 Loading first {n_rows} rows using pandas...")
    
    try:
        # Pandas can read first N rows directly from URL
        df = pd.read_csv(url, nrows=n_rows)
        
        print(f"✅ Loaded {len(df)} rows with {len(df.columns)} columns")
        print(f"📋 Columns: {list(df.columns)}")
        
        # Convert to HuggingFace Dataset
        dataset = Dataset.from_pandas(df)
        
        print("🎉 Success!")
        return dataset
        
    except Exception as e:
        print(f"❌ Pandas approach failed: {e}")
        return None

# Main execution
url = "https://nakhun-chumpolsathien.oss-us-west-1.aliyuncs.com/thaisum/thaisum.csv"

print("🌟 THAISUM - FIRST 500 ROWS LOADER")
print("="*50)

# Try the simplest approach first
print("🚀 Attempting simple pandas approach...")
ds = load_simple_pandas(url, 500)

if ds is None:
    print("\n🔄 Trying streaming approach...")
    ds = load_first_500_samples(url)

# Result
if ds is not None:
    print(f"\n🎉 SUCCESS!")
    print(f"📊 Dataset loaded: {len(ds)} samples")
    print(f"🔑 Available fields: {list(ds.features.keys())}")
    print(f"💡 Ready to use!")
    
else:
    print("\n❌ Failed to load dataset")

🌟 THAISUM - FIRST 500 ROWS LOADER
🚀 Attempting simple pandas approach...
📥 Loading first 500 rows using pandas...
✅ Loaded 500 rows with 6 columns
📋 Columns: ['title', 'body', 'summary', 'type', 'tags', 'url']
🎉 Success!

🎉 SUCCESS!
📊 Dataset loaded: 500 samples
🔑 Available fields: ['title', 'body', 'summary', 'type', 'tags', 'url']
💡 Ready to use!


In [10]:
ds[0]

{'title': ' วิษณุ ยันโรดแม็ปเดิม ตอบไม่ถูกเวลาเลือกตั้ง ต้องรอ รธน.ประกาศใช้',
 'body': 'เมื่อวันที่ 6 ม.ค.60 ที่ทำเนียบรัฐบาล นายวิษณุ เครืองาม รองนายกรัฐมนตรี กล่าวถึงกรณี ที่ นายสุรชัย เลี้ยงบุญเลิศชัย รองประธานสภานิติบัญญัติแห่งชาติ (สนช.) ออกมาระบุว่า การเลือกตั้งจะถูกเลื่อนออกไปถึงปี 2561 ว่า ขอให้ไปสอบถามกับ สนช. แต่เชื่อว่าคงไม่กล้าพูดอีก เพราะทำให้คนเข้าใจผิด ซึ่งที่ สนช.พูดเนื่องจากผูกกับกฎหมายของกรรมการร่างรัฐธรรมนูญ(กรธ.) ตนจึงไม่ขอวิพากษ์วิจารณ์ แต่รัฐบาลยืนยันว่ายังเดินตามโรดแม็ป ซึ่งโรดแม็ปมองได้สองแบบ คือ มีลำดับขั้นตอนและการกำหนดช่วงเวลา โดยเริ่มต้นจากการประกาศใช้รัฐธรรมนูญ แต่ขณะนี้รัฐธรรมนูญยังไม่ประกาศใช้ จึงยังเริ่มนับหนึ่งไม่ถูก จากนั้นเข้าสู่ขั้นตอนการร่างกฎหมายประกอบร่างรัฐธรรมนูญหรือกฎหมายลูก ภายใน 240 วัน ก่อนจะส่งกลับให้ สนช.พิจารณา ภายใน 2 เดือน\xa0,นายวิษณุ กล่าวต่อว่า หากมีการแก้ไขก็จะมีการพิจารณาร่วมกับ กรธ.อีก 1 เดือน ก่อนนำขึ้นทูลเกล้าฯ ทรงลงพระปรมาภิไธย ภายใน 90 วัน และจะเข้าสู่การเลือกตั้งภายในระยะเวลา 5 เดือน ซึ่งทั้งหมดนี้คือโรดแม็ปที่ยังเป็นแบบเดิม

In [13]:
# Tokenize
from attacut import Tokenizer

atta = Tokenizer(model="attacut-sc")

def tokenize_batch(examples):
    """Process multiple texts at once"""
    tokenized_bodies = []
    for body in examples['body']:
        if body:
            tokens = atta.tokenize(body)
            tokenized_bodies.append(" ".join(tokens))
        else:
            tokenized_bodies.append("")
    return {'tokenized_body': tokenized_bodies}

ds_tok = ds.map(
    tokenize_batch,
    batched=True,          
    batch_size=20,  
    remove_columns=['body'],
    desc="Tokenizing Thai text"
)

Tokenizing Thai text:   0%|          | 0/500 [00:00<?, ? examples/s]

In [15]:
ds_tok[0]['tokenized_body']

'เมื่อ วัน ที่   6   ม. ค. 60   ที่ ทำเนียบรัฐบาล   นายวิษณุ เครืองาม   รอง นายก รัฐมนตรี   กล่าว ถึง กรณี   ที่   นายสุรชัย เลี้ยงบุญเลิศชัย   รอง ประธาน สภานิติบัญญัติแห่งชาติ   ( สนช. )   ออก มา ระบุ ว่า   การ เลือกตั้ง จะ ถูก เลื่อน ออก ไป ถึง ปี   2561   ว่า   ขอ ให้ ไป สอบถาม กับ   สนช.   แต่ เชื่อ ว่า คง ไม่ กล้า พูด อีก   เพราะ ทำ ให้ คน เข้าใจ ผิด   ซึ่ง ที่   สนช. พูด เนื่อง จาก ผูก กับ กฎหมาย ของ กรรมการ ร่าง รัฐธรรมนูญ ( กร ธ. )   ตน จึง ไม่ ขอ วิพากษ์วิจารณ์   แต่ รัฐบาล ยืนยัน ว่า ยัง เดิน ตาม โรด แม็ป   ซึ่ง โรด แม็ป มอง ได้ สอง แบบ   คือ   มี ลำดับ ขั้นตอน และ การ กำหนด ช่วง เวลา   โดย เริ่มต้น จาก การ ประกาศ ใช้ รัฐธรรมนูญ   แต่ ขณะ นี้ รัฐธรรมนูญ ยัง ไม่ ประกาศ ใช้   จึง ยัง เริ่ม นับ หนึ่ง ไม่ ถูก   จาก นั้น เข้า สู่ ขั้นตอน การ ร่าง กฎหมาย ประกอบ ร่าง รัฐธรรมนูญ หรือ กฎหมาย ลูก   ภาย ใน   240   วัน   ก่อน จะ ส่ง กลับ ให้   สนช. พิจารณา   ภาย ใน   2   เดือน \xa0 , นายวิษณุ   กล่าว ต่อ ว่า   หาก มี การ แก้ไข ก็ จะ มี การ พิจารณา ร่วม กับ   กร ธ. อีก   1   เดือน   ก่อน

In [26]:
# Import to Knowledge Base
from langchain.docstore.document import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["body"], metadata={"source": doc["url"]})
    for doc in tqdm(ds)
]

100%|██████████| 500/500 [00:00<00:00, 6589.97it/s]


In [27]:
# Split sentence
from langchain.text_splitter import RecursiveCharacterTextSplitter

MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    add_start_index=True,
    strip_whitespace=True,
    separators=MARKDOWN_SEPARATORS,
)

docs_processed = []
for doc in RAW_KNOWLEDGE_BASE:
    docs_processed += text_splitter.split_documents([doc])

In [46]:
from sentence_transformers import SentenceTransformer

sentence_embedding_path = "sentence-transformers/distiluse-base-multilingual-cased-v2"

print(
    f"Model's maximum sequence length: {SentenceTransformer(sentence_embedding_path).max_seq_length}"
)

Model's maximum sequence length: 128


In [47]:
# Split document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
from tqdm import tqdm

EMBEDDING_MODEL_NAME = sentence_embedding_path


def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: Optional[str] = EMBEDDING_MODEL_NAME,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of maximum size `chunk_size` tokens and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=MARKDOWN_SEPARATORS,
    )

    docs_processed = []
    for doc in tqdm(knowledge_base):
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in tqdm(docs_processed):
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique


docs_processed = split_documents(
    512, 
    RAW_KNOWLEDGE_BASE,
    tokenizer_name=EMBEDDING_MODEL_NAME,
)

100%|██████████| 2061/2061 [00:00<00:00, 710717.80it/s]


In [29]:
# Embedding Knowledge Base
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # set True for cosine similarity
)

KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

In [48]:
# embed a user query in the same space
user_query = "นายวิษณุ เครืองาม พูดถึง นายสุรชัย เลี้ยงบุญเลิศชัย เรื่องอะไร"
# user_query = thai_tok(user_query)
query_vector = embedding_model.embed_query(user_query)

print(f"\nStarting retrieval for {user_query=}...")
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=5)
print(
    "\n==================================Top document=================================="
)
print(retrieved_docs[0].page_content)
print("==================================Metadata==================================")
print(retrieved_docs[0].metadata)


Starting retrieval for user_query='นายวิษณุ เครืองาม พูดถึง นายสุรชัย เลี้ยงบุญเลิศชัย เรื่องอะไร'...

==================================Top document==================================
เผื่อมีน้องไม่ทันใช้? ไม่เป็นไร ใช้ลูกคนอื่นก็ได้ครับผม เห็นว่ามีส่วนร่วมในการเล่นเรื่องเจ้าบ้านเจ้าเรือนด้วย? ผมก็ได้ยินอยู่นะ แต่ว่ายังไม่ได้รับการคอนเฟิร์มกับทางช่อง ผมก็เพิ่งปิดเรื่องหัวใจปฐพี และนางสาวทองสร้อย เค้าได้มีการติดต่อมาทางช่องไหม? คือถ้าโดยหลักการแล้ว ผมเป็นนักแสดงช่อง 3 ก็ยังไม่ได้รับคำสั่งจากทางช่อง ผมก็ได้ยินนะครับแต่ผมยังไม่ได้รับคำสั่งจากทางช่อง ถ้าได้เล่นจริงๆ ก็จะเป็นพระรอง เพราะพี่ติ๊ก เจษฎาภรณ์เป็นพระเอก ผมไม่รู้สึกอะไร เพราะผมเป็นนักแสดง ผมเล่นได้ทุกบท ไม่เกี่ยงบทใช่ไหม? ผมก็ไม่เคยเลือกอยู่แล้วนะ ตั้งแต่เป็นนักแสดงมา ก็จะเป็นทางช่องเลือกให้หมดทุกอย่าง เราเป็นพระเอกมาตลอด แต่เรื่องนี้มีพระเอกอีกคนนึง? ผมก็เคยเล่นพระเอก 3 คนก็เคย 2 คนก็เคย มันไม่ใช่ปัญหาหรอกครับ ประเด็นมันไม่ได้อยู่ตรงนั้น อย่าไปทำให้มันมีประเด็นเลย
==================================Metadata=======================

In [32]:
# Add bos
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Model and tokenizer loading
READER_MODEL_NAME = "SeaLLMs/SeaLLM-7B-v2.5"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(
    READER_MODEL_NAME, quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

# Ensure '<bos>' token is added if not handled by the tokenizer
def ensure_bos(token_ids):
    bos_token_id = tokenizer.bos_token_id
    if token_ids[0] != bos_token_id:
        return [bos_token_id] + token_ids
    return token_ids
    
# Define LLM model
READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    # repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=200,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id
)

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

In [36]:
# Define prompt
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]

RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

<bos><|im_start|>system
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.<eos>
<|im_start|>user
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}<eos>
<|im_start|>assistant



In [37]:
retrieved_docs_text = [
    doc.page_content for doc in retrieved_docs
]  # we only need the text of the documents
context = "\nExtracted documents:\n"
context += "".join(
    [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)]
)

final_prompt = RAG_PROMPT_TEMPLATE.format(
    question=user_query, context=context
)

# Redact an answer
answer = READER_LLM(final_prompt)[0]["generated_text"]
print(answer)

นายวิษณุ เครืองาม พูดถึง นายสุรชัย เลี้ยงบุญเลิศชัย เรื่องการที่นายสุรชัย เลี้ยงบุญเลิศชัย มีการติดต่อกับทางช่อง 3 เพื่อขอรับคำสั่งในการเล่นเรื่องเจ้าบ้านเจ้าเรือน แต่ยังไม่ได้รับการคอนเฟิร์มกับทางช่อง และยังไม่ได้รับคำสั่งจากทางช่อง


In [38]:
# print(context)
print(final_prompt)

<bos><|im_start|>system
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.<eos>
<|im_start|>user
Context:

Extracted documents:
Document 0:::
เผื่อมีน้องไม่ทันใช้? ไม่เป็นไร ใช้ลูกคนอื่นก็ได้ครับผม เห็นว่ามีส่วนร่วมในการเล่นเรื่องเจ้าบ้านเจ้าเรือนด้วย? ผมก็ได้ยินอยู่นะ แต่ว่ายังไม่ได้รับการคอนเฟิร์มกับทางช่อง ผมก็เพิ่งปิดเรื่องหัวใจปฐพี และนางสาวทองสร้อย เค้าได้มีการติดต่อมาทางช่องไหม? คือถ้าโดยหลักการแล้ว ผมเป็นนักแสดงช่อง 3 ก็ยังไม่ได้รับคำสั่งจากทางช่อง ผมก็ได้ยินนะครับแต่ผมยังไม่ได้รับคำสั่งจากทางช่อง ถ้าได้เล่นจริงๆ ก็จะเป็นพระรอง เพราะพี่ติ๊ก เจษฎาภรณ์เป็นพระเอก ผมไม่รู้สึกอะไร เพราะผมเป็นนักแสดง ผมเล่นได้ทุกบท ไม่เกี่ยงบทใช่ไหม? ผมก็ไม่เคยเลือกอยู่แล้วนะ ตั้งแต่เป็นนักแสดงมา ก็จะเป็นทางช่องเลือกให้หมดทุกอย่าง เราเป็นพร

In [40]:
# Rerank answers
from ragatouille import RAGPretrainedModel

RERANKER = RAGPretrainedModel.from_pretrained("antoinelouis/colbert-xm") # multilingual

def answer_with_rag(
    question: str,
    llm: Pipeline,
    knowledge_index: FAISS,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 5,
) -> Tuple[str, List[LangchainDocument]]:
    # Gather documents with retriever
    print("=> Retrieving documents...")
    relevant_docs = knowledge_index.similarity_search(
        query=question, k=num_retrieved_docs
    )
    relevant_docs = [doc.page_content for doc in relevant_docs]  # keep only the text

    # Optionally rerank results
    if reranker:
        print("=> Reranking documents...")
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join(
        [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)]
    )

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    print("=> Generating answer...")
    answer = llm(final_prompt)[0]["generated_text"]

    return answer, relevant_docs

In [41]:
# Final
question = "นายวิษณุ เครืองาม พูดถึง นายสุรชัย เลี้ยงบุญเลิศชัย เรื่องอะไร"

answer, relevant_docs = answer_with_rag(
    question, READER_LLM, KNOWLEDGE_VECTOR_DATABASE, reranker=RERANKER
)

print("==================================Answer==================================")
print(f"{answer}")
print("==================================Source docs==================================")
for i, doc in enumerate(relevant_docs):
    print(f"Document {i}------------------------------------------------------------")
    print(doc)

=> Retrieving documents...


/root/miniconda3/envs/py3.11/lib/python3.11/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


=> Reranking documents...


100%|██████████| 1/1 [00:00<00:00, 14.47it/s]

=> Generating answer...


==================================Answer==================================
นายวิษณุ เครืองามไม่ได้พูดถึงนายสุรชัย เลี้ยงบุญเลิศชัยในบริบทที่ให้มา
==================================Source docs==================================
Document 0------------------------------------------------------------
สนช.บางส่วนที่ยังติดใจขอสงวนคำแปรญัตติไว้อภิปรายในที่ประชุม สนช. เนื่องจากอยากให้ กกต.ชุดปัจจุบันทำหน้าที่ต่อไปจนครบวาระ,นายสมชัย ศรีสุทธิยากร กกต.ด้านบริหารกลาง กล่าวว่า หากให้ภาพอนาคตเลวร้ายสุดคือให้ กกต.ขาดคุณสมบัติตามรัฐธรรมนูญใหม่ คนแรกที่ขาดคุณสมบัติคือนายประวิช รัตนเพียร กกต.ด้านการมีส่วนร่วม ส่วนตนต้องยื่นหลักฐานทำงานภาคประชาสังคมหากกรรมการสรรหาไม่รับฟังจะพ้นจากตำแหน่ง อีก 2 คนจะหมดวาระเพราะอายุครบ 70 ปี คือนายบุญส่ง น้อยโสภณ เดือน ก.ค.61 นายศุภชัย เดือน ก.พ.62 เหลือนายธีรวัฒน์ ธีรโรจน์วิทย์ กกต.ด้านพรรคการเมืองคนเดียว ถ้านายธีรวัฒน์ ถูกตรวจสอบด้านจริยธรรม อาจทำให้การเลือกตั้งหลังเดือน ก.พ.62 อยู่ในมือ กกต.ใหม่ทั้ง 7 คน ส่วนภาพดีสุดคือ สนช.ยึดคุณสมบัติตามรัฐธรรมนูญเดิมให้
Document 1----